Харламова Дарья Сергеевна, БКЛ-212
Домашнее задание 1.

В данном блокноте мы займемся классификацией отзывов на положительные и отрицательные с помощью алгоритмического кода. В качестве "подопытного" выступит известный сайт "Кинопоиск", а разбирать мы будем рецензии на фильмы. 

Для начала импортируем все необходимые нам библиотеки:

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import itertools
from string import punctuation
from collections import Counter
from sklearn.metrics import accuracy_score

Первым шагом займемся парсингом данных. Создадим сессию. Выяснив, что "Кинопоиск" быть подопытным не хочет, а вместо этого хочет банить наши запросы как подозрительные и подсовывать нам капчу, притворимся браузером.

In [2]:
session = requests.session()
ua = UserAgent(browsers = ["chrome", "edge", "firefox", "safari"])
adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=10)
session.mount('https://', adapter)

Напишем функцию, которая будет принимать на вход URL-адрес страницы с отзывами, а возвращать - список хороших и плохих отзывов, которые присутствуют на странице. 

In [3]:
def get_good_bad_reviews(url):
    positive_all_entries = []
    negative_all_entries = []
    response = session.get(url, headers={'User-Agent': ua.random})
    soup = BeautifulSoup(response.text, 'html.parser')
    positive_all_entries = soup.find_all('div', {'class': 'response good'})
    negative_all_entries = soup.find_all('div', {'class': 'response bad'})
    return positive_all_entries, negative_all_entries

Изначально мы хотели собрать рецензии на разные фильмы и проитерироваться по разным URL - но Кинопоиску кажется это подозрительным, поэтому пока ограничимся одним. Фильм нужен популярный (чтобы набрать большое количество отзывов сразу) и достаточно спорный, чтобы с него можно было собрать примерно равное количество положительных и отрицательных рецензий. Выбор на самом деле очевиден - "Сумерки"!

Для быстрых модификаций - для сбора рецензий по большему количеству фильмов достаточно добавить URL-адреса "Кинопоиска" в список url_working (но обычно к обработке второй ссылки "Кинопоиск" уже выкидывает экран с капчей и сбегает, хохоча).

In [4]:
url_working = ["https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/all/perpage/200/"]

In [5]:
pos_list = []
neg_list = []
control = []
for link in url_working:
    pair = get_good_bad_reviews(link)
    pos_list.append(pair[0])
    neg_list.append(pair[1])

Отобрав у "Кинопоиска" рецензии, соберем положительные и отрицательные рецензии в один список и посмотрим на масштаб нашего "улова".

In [6]:
pos_parse = list(itertools.chain.from_iterable(pos_list))
neg_parse = list(itertools.chain.from_iterable(neg_list))

In [7]:
print(len(pos_parse))
print(len(neg_parse))

75
76


Бинго - у нас практически равное количество положительных и отрицательных рецензий! Теперь отберем из них контрольную группу, оформив ее в словарик, где ключи - рецензии, а значения обозначают, положительной или отрицательной является рецензия.

In [8]:
control_parse = {}
for revieows in pos_parse[60:76]:
    control_parse[revieows] = 'Positive'
del pos_parse[60:76]
for revieows in neg_parse[60:77]:
    control_parse[revieows] = 'Negative'
del neg_parse[60:77]

Посмотрим на итоговое количество рецензий, которые оказались в контрольной группе и в группах, по которым мы будем собирать словарики:

In [9]:
print(len(control_parse))
print(len(pos_parse))
print(len(neg_parse))

31
60
60


Теперь начнем, собственно, собирание словариков. Для лемматизации будем пользоваться Mystem - создадим его объект. Еще в процессе очистки текста уберем из него стоп-слова: для этого импортируем из NLTK список русских стоп-слов.

In [10]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

Зададим функцию, которая будет очищать нам текст. На вход она получает обработанный "супом" HTML-объект и флажок, который обозначает, интересует ли нас целиковый текст. 

Если передать 1, то функция отдаст содержащийся в "супе" текст рецензии в немного причесанном виде

Если передать 0, то функция вернет лемматизированный и токенизированный текст, по возможности очищенный от пунктуации. 

Для работы нам нужнен только 0, но возможность передать единицу присутствует для удобства демонстрации.

In [11]:
def process_review(review_soup, print_text):
    word_list = []
    review = str(review_soup.text)
    review = review[review.find('звёзды')+6:review.rfind('прямая ссылка')]
    review = review.strip()
    review = review.replace('\n', ' ')
    review = review.replace('\r', '')
    if print_text == 1:
        return review
    if print_text == 0:
        review = mystem.lemmatize(review.lower())
        for token in review:
            token = token.replace(' ', '')
            if token != ' ' and token not in russian_stopwords and token not in punctuation:
                word_list.append(token)
        return word_list

Пользуясь данной функцией, соберем для больших списка со всеми словами из положительных и отрицательных рецензий.

In [12]:
pos_words = []
neg_words = []
for rev_g in pos_parse:
    pos_words.append(process_review(rev_g, 0))
for rev_n in neg_parse:
    neg_words.append(process_review(rev_n, 0))

Теперь напишем еще одну функцию - она будет получать на вход список из списков слов из каждой рецензии, соединять их в один, и сводить данные о частотности каждого слова в словарик, который и будет возвращаться.

In [13]:
def set_freq_list(words_list):
    words_list = list(itertools.chain.from_iterable(words_list))
    freqs = {}
    for word in words_list:
        freqs[word] = freqs.get(word, 0) + 1
    return(freqs)

Получим словарики и отфильтруем их по частотности. Для того, чтобы быть достаточно объективными, будем учитывать слова, которые встретились суммарно не менее 10 раз:

In [14]:
pos_freq_words = {}
for i in set_freq_list(pos_words).items():
    if i[1] > 10:
        pos_freq_words[i[0]] = i[1]

In [15]:
neg_freq_words = {}
for j in set_freq_list(neg_words).items():
    if j[1] > 10:
        neg_freq_words[j[0]] = j[1]

Теперь из списка ключей наших словарей соберем множество.

In [16]:
pos_words_set = set(pos_freq_words.keys())
neg_words_set = set(neg_freq_words.keys())

Сохраним в переменные `pos_test` и `neg_test` элементы, которых нет в другом множестве:

In [17]:
pos_test = pos_words_set.difference(neg_words_set)
neg_test = neg_words_set.difference(pos_words_set)

Теперь напишем функцию оценивания. На вход она должна принимать лемматизированный список слов, которые есть в рецензии. Итерируясь по этим словам, она ведет два счетчика, значение которых увеличивается на единицу с каждым словом, которое есть во множестве `pos_test` или `neg_test`. В зависимости от значений счетчиков функция принимает решение о том, является ли отзыв положительным или отрицательным.

In [18]:
def evaluate_stuff(rev):
    pos_score = 0
    neg_score = 0
    for slovo in rev:
        if slovo in pos_test:
            pos_score += 1
        if slovo in neg_test:
            neg_score += 1
    if pos_score > neg_score:
        return 'Positive'
    if neg_score > pos_score:
        return 'Negative'
    if pos_score == neg_score:
        return 'Neutral'

Прогоним наши тексты из контрольной группы через функции `process_review` и `evaluate_stuff`.

In [19]:
test = []
for texts in control_parse:
    test.append(evaluate_stuff(process_review(texts, 0)))
test

['Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive']

Получим список истинных значений для контрольной группы, которые мы раньше сохранили в значениях словаря `control_parse`:

In [20]:
true = control_parse.values()
true

dict_values(['Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative'])

Посчитаем качество с помощью метрики `accuracy`:

In [21]:
accuracy_score(list(true), test)

0.7419354838709677

Итак, мы видим, что мы достигли сносной точности, но нам всё ещё есть куда расти. Например, можно попробовать учитывать более частотные слова с большим весом, чем менее частотные. Еще можно попробовать учитывать не только отдельные слова, а словосочетания из 2-3 элементов: это тоже может привести к большей точности.

Попробуем в самом примитивном виде реализовать концепцию с весами. Сначала посмотрим, какие частоты у нас максимальные, обратившись к словарям с частотностью.

In [22]:
max(list(pos_freq_words.values()))

511

In [23]:
max(list(neg_freq_words.values()))

372

Теперь разобьем все слова на три класса по частотности. Допустим, самыми частотными будут считаться слова, которые встретились больше 150 раз, средними по частотности - слова, которые встретились от 50 до 150 раз, наименее частотными - слова, которые встретились меньше 50 раз. Распределим по этим категориям слова из оценочного множества:

In [24]:
pos_test_max = []
pos_test_mid = []
pos_test_min = []
for w in pos_test:
    if pos_freq_words[w] > 150:
        pos_test_max.append(w)
    elif 50 < pos_freq_words[w] < 151:
        pos_test_mid.append(w)
    elif pos_freq_words[w] < 51:
        pos_test_min.append(w)

Проверим, что никого не потеряли:

In [25]:
print(len(pos_test))
print(len(pos_test_max) + len(pos_test_mid) + len(pos_test_min))

94
94


Повторим для негативных слов:

In [26]:
neg_test_max = []
neg_test_mid = []
neg_test_min = []
for w in neg_test:
    if neg_freq_words[w] > 150:
        neg_test_max.append(w)
    elif 50 < neg_freq_words[w] < 151:
        neg_test_mid.append(w)
    elif neg_freq_words[w] < 49:
        neg_test_min.append(w)

In [27]:
print(len(neg_test))
print(len(neg_test_max) + len(neg_test_mid) + len(neg_test_min))

72
72


Теперь зададим нашу функцию. Она очень похожа на функцию `evaluate_stuff`, но в зависимости от класса частотности, в который входит слово, приписывает положительному и отрицательному счетам разные количества очков. Так как предыдущий алгоритм скорее кренился в позитивные отзывы, немного усилим позиции негативных слов:

In [28]:
def evaluate_stuff_w(rev):
    pos_score = 0
    neg_score = 0
    for slovo in rev:
        if slovo in pos_test_max:
            pos_score += 3
        elif slovo in pos_test_mid:
            pos_score += 2
        elif slovo in pos_test_min:
            pos_score += 1
        elif slovo in neg_test_max:
            neg_score += 3
        elif slovo in neg_test_mid:
            neg_score += 3
        elif slovo in neg_test_min:
            neg_score += 2
    if pos_score > neg_score:
        return 'Positive'
    if neg_score > pos_score:
        return 'Negative'
    if pos_score == neg_score:
        return 'Neutral'

Проверим это наивное добавление на практике:

In [29]:
test = []
for texts in control_parse:
    test.append(evaluate_stuff_w(process_review(texts, 0)))
test

['Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative']

In [30]:
true = control_parse.values()
true

dict_values(['Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative'])

In [31]:
accuracy_score(list(true), test)

0.8064516129032258

Видим, что пусть наше добавление и было простым и невинным, на данной выборке точность оценивания немного подросла.